# 1.数据介绍
数据包括五个不同的文件，四个文件包括用户的id和信息
* 在train.csv中我们可以看到用户的id和这个用户是否流失
* 在transactions.csv文件中是用户的交易记录和此交易是否被取消
* user_logs.csv文件是用户的行为日志文件，包括用户听了什么歌听了多久等信息
* members.csv文件是用户信息文件，包括用户年龄、城市等
* sample_submission_zero.csv文件是测试集

# 2.读取数据
因为个别数据集过大，所以需要使用特殊的方法，对于读取大文件，一般办法有：
* 分块读取，可以使用pandas的chunksize来分块读取（但用户日志文件将近28G，仍人需要大量内存，16G比较稳妥）
* 使用Spark和Hadoop
* 借助数据库

这里我采用的是pandas将大文件分割成数个小文件，逐个击破

In [1]:
import pandas as pd

In [2]:
# chunksize = pd.read_csv('B:\PythonPratices\kaggle\liushi/user_logs.csv', chunksize=5000000)
# i = 0
# for chunk in chunksize:
#     i += 1
#     try:
#         chunk.to_csv('B:/PythonPratices/kaggle/liushi/user_log_split_' + str(i)+'.csv')
#         print(str(i)+"号文件写入成功") 
#     except Exception as e:
#         print(str(i)+"号文件写入失败")   

In [3]:
# chunksize = pd.read_csv('B:\PythonPratices\kaggle\liushi/transactions.csv', chunksize=500000)
# transactions = pd.DataFrame()
# for chunk in chunksize:
#     transactions = pd.concat([chunk, transactions], ignore_index=True)
#     pd.merge(members, transactions.drop_duplicates(), on='msno', how='left')

# 3.文件数据结构和内容探索

## 3.1 train.csv

In [5]:
train_data = pd.read_csv("G:\PythonProjects\KaggleOrOthersJourney\WSDM-KKBox'sChurnPredictionChallenge\data/train.csv")

In [6]:
train_data.count()

msno        992931
is_churn    992931
dtype: int64

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992931 entries, 0 to 992930
Data columns (total 2 columns):
msno        992931 non-null object
is_churn    992931 non-null int64
dtypes: int64(1), object(1)
memory usage: 15.2+ MB


In [8]:
train_data.head(10)

,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1
5,GBy8qSz16X5iYWD+3CMxv/Hm6OPSrXBYtmbnlRtknW0=,1
6,lYLh7TdkWpIoQs3i3o6mIjLH8/IEgMWP9r7OpsLX0Vo=,1
7,T0FF6lumjKcqEO0O+tUH2ytc+Kb9EkeaLzcVUiTr1aE=,1
8,Nb1ZGEmagQeba5E+nQj8VlQoWl+8SFmLZu+Y8ytIamw=,1
9,MkuWz0Nq6/Oq5fKqRddWL7oh2SLUSRe3/g+XmAWqW1Q=,1


In [14]:
train_data['is_churn'].value_counts()

0    929460
1     63471
Name: is_churn, dtype: int64

## 通过上面分析我们可以发现：
1. 文件没有缺失数据，总共992931条数据
1. train.csv包括用户id（加密的字符串）和是否流失is_churn两个属性
2. is_churn包括0和1两个值，其中0值929460条，1值63471条（非常小的比例）

## 3.2 members.csv

In [15]:
members_data = pd.read_csv("G:\PythonProjects\KaggleOrOthersJourney\WSDM-KKBox'sChurnPredictionChallenge\data/members.csv")

In [16]:
members_data.head(10)

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,URiXrfYPzHAlk+7+n7BOMl9G+T7g8JmrSnT/BU8GmEo=,1,0,NaN,9,20150525,20150526
1,U1q0qCqK/lDMTD2kN8G9OXMtfuvLCey20OAIPOvXXGQ=,1,0,NaN,4,20161221,20161224
2,W6M2H2kAoN9ahfDYKo3J6tmsJRAeuFc9wl1cau5VL1Q=,1,0,NaN,4,20160306,20160309
3,1qE5+cN7CUyC+KFH6gBZzMWmM1QpIVW6A43BEm98I/w=,5,17,female,4,20161031,20161107
4,SeAnaZPI+tFdAt+r3lZt/B8PgTp7bcG/1os39u4pLxs=,1,0,NaN,4,20170202,20170205
5,iSJMDbqVrpRf7DRQoCINa6CzxpEIkHj91X5oqpReC3k=,1,0,NaN,4,20160215,20160218
6,NNYKvjuUcXHLi40gmdfpVfdlNOuRxQ1uGYVp45Ce/eY=,1,0,NaN,9,20150719,20150720
7,TA85iK5eP14OHddABVsSlwi4EKTe9sxMBp+9nWsdLq8=,1,40,male,3,20150606,20150615
8,75rp8sCaeatezkb6c0I133UakcSDuKcc5ia0b0Q+JS4=,1,24,male,3,20151129,20170925
9,1Nqjt8ykQceNOmuRrh9bcgvicDfLlj+jkgx7+Dd0Rk0=,1,0,NaN,7,20160410,20170909


In [17]:
members_data.count()

msno                      5116194
city                      5116194
bd                        5116194
gender                    1761416
registered_via            5116194
registration_init_time    5116194
expiration_date           5116194
dtype: int64

In [18]:
members_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5116194 entries, 0 to 5116193
Data columns (total 7 columns):
msno                      object
city                      int64
bd                        int64
gender                    object
registered_via            int64
registration_init_time    int64
expiration_date           int64
dtypes: int64(5), object(2)
memory usage: 273.2+ MB


In [19]:
members_data.describe()

,city,bd,registered_via,registration_init_time,expiration_date
count,5.116194e+06,5.116194e+06,5.116194e+06,5.116194e+06,5.116194e+06
mean,4.019725e+00,9.772362e+00,5.248127e+00,2.014631e+07,2.015981e+07
std,5.590289e+00,1.795875e+01,2.260226e+00,2.308540e+04,1.254307e+04
min,1.000000e+00,-6.998000e+03,3.000000e+00,2.004033e+07,1.970010e+07
25%,1.000000e+00,0.000000e+00,4.000000e+00,2.014103e+07,2.015111e+07
50%,1.000000e+00,0.000000e+00,4.000000e+00,2.015111e+07,2.016070e+07
75%,5.000000e+00,2.100000e+01,7.000000e+00,2.016061e+07,2.017061e+07
max,2.200000e+01,2.015000e+03,1.600000e+01,2.017071e+07,2.100010e+07


In [21]:
members_data.bd.value_counts()

0      3421570
21       85143
20       85132
22       84152
23       77151
27       75995
24       74095
26       73604
19       72257
25       70659
18       67422
28       64597
17       63468
29       61338
30       53269
32       49678
31       48694
33       44301
34       42551
35       39978
37       39558
36       39094
16       38479
38       31386
39       28166
40       26438
41       24120
42       20448
43       18317
44       16658
        ...   
462          1
519          1
573          1
584          1
940          1
939          1
935          1
934          1
929          1
926          1
925          1
923          1
922          1
920          1
919          1
906          1
837          1
827          1
826          1
822          1
821          1
810          1
806          1
793          1
786          1
778          1
743          1
689          1
649          1
323          1
Name: bd, Length: 326, dtype: int64

In [31]:
members_data.gender.value_counts(dropna=False) # 不忽略NaN值

NaN       3354778
male       902851
female     858565
Name: gender, dtype: int64

In [33]:
members_data['registered_via'].value_counts()

4     2351944
9     1063941
3     1015513
7      680365
13       4246
16        175
10         10
Name: registered_via, dtype: int64

In [34]:
members_data['city'].value_counts()

1     3541998
5      309458
13     266059
4      196290
22     167575
15     153272
6      107237
14      71879
12      51956
9       38955
11      36935
8       35846
18      30006
10      25674
21      23896
17      21743
3       21198
7        8486
16       3814
20       3163
19        754
Name: city, dtype: int64

### 通过上面分析我们可以发现：
1. 除了性别以外，其他字段没有缺失值，性别存在大量缺失值。数据一共5116194条
2. 年龄存在很多异常值，官网给的说明是年龄异常值范围在-7000~2015，需要自己处理，通过统计发现年龄为0的最多（默认值？），还有很多100以上的，但每个数量都很少
3. 通过统计发现，性别上男女比例差不多，但是Nan值占了绝大多数
4. 用户的注册方式以“4”最多，注册方式映射为数字3，4，7，9，10，13，16
5. 城市一共21个，被映射为数字，但是没有“2”
6. registration_init_time和expiration_date是时间列，字面意思理解是注册时间和终结时间（流失时间）

## 3.3 transaction.csv

In [36]:
chunksize = pd.read_csv("G:\PythonProjects\KaggleOrOthersJourney\WSDM-KKBox'sChurnPredictionChallenge\data/transactions.csv", chunksize=1000000)
transactions = pd.DataFrame()
for chunk in chunksize:
    transactions = pd.concat([chunk, transactions], ignore_index=True)
    # pd.merge(members, transactions.drop_duplicates(), on='msno', how='left')

In [37]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21547746 entries, 0 to 21547745
Data columns (total 9 columns):
msno                      object
payment_method_id         int64
payment_plan_days         int64
plan_list_price           int64
actual_amount_paid        int64
is_auto_renew             int64
transaction_date          int64
membership_expire_date    int64
is_cancel                 int64
dtypes: int64(8), object(1)
memory usage: 1.4+ GB


In [38]:
transactions.count()

msno                      21547746
payment_method_id         21547746
payment_plan_days         21547746
plan_list_price           21547746
actual_amount_paid        21547746
is_auto_renew             21547746
transaction_date          21547746
membership_expire_date    21547746
is_cancel                 21547746
dtype: int64

In [39]:
transactions.describe()

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
count,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07
mean,3.893310e+01,3.133906e+01,1.398850e+02,1.419873e+02,8.519661e-01,2.015760e+07,2.015862e+07,3.976523e-02
std,3.507936e+00,3.035649e+01,1.309647e+02,1.324824e+02,3.551336e-01,6.155979e+03,7.755753e+03,1.954072e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.015010e+07,1.970010e+07,0.000000e+00
25%,3.800000e+01,3.000000e+01,9.900000e+01,9.900000e+01,1.000000e+00,2.015091e+07,2.015102e+07,0.000000e+00
50%,4.100000e+01,3.000000e+01,1.490000e+02,1.490000e+02,1.000000e+00,2.016032e+07,2.016043e+07,0.000000e+00
75%,4.100000e+01,3.000000e+01,1.490000e+02,1.490000e+02,1.000000e+00,2.016092e+07,2.016103e+07,0.000000e+00
max,4.100000e+01,4.500000e+02,2.000000e+03,2.000000e+03,1.000000e+00,2.017023e+07,2.017033e+07,1.000000e+00


In [40]:
transactions.head(10)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,cXqexun2tGbJEoQJER2vamxodTOX96yFUt7GTLLM+sY=,36,30,180,180,0,20170112,20170211,0
1,7nN+f8W03OxhlRgf6uhkNsLLI2pb4GtMfUWbLcdi59A=,40,30,149,149,1,20170112,20170213,0
2,yOPXNAn5GWYaePY+PoYIfdfT/dynwAGu0lecyZfcveM=,41,30,99,99,1,20170112,20170211,0
3,9S2AZpj2NRaW/SxXjP10joGQhge+jBaPvdT+Bs7QshA=,41,30,129,129,1,20170119,20170313,0
4,woF2/69OJG4qfW/OlAXUyr575zY8ubKKDmJ7JhH28ZA=,41,30,149,149,1,20170119,20170219,0
5,JZq7hU4BMXLDwbimldrmKZXyWSmxtmHhVNh204ihlts=,41,30,99,99,1,20170119,20170219,0
6,xabOd42/ZOAu1cdvWydMZF39oIFp9SLgYUxwL8S4bWA=,41,30,99,99,1,20170119,20170219,0
7,8mjY0GHBt7fkzJutgUMk9tqon8XoavxpG8KdasJz8fc=,41,30,99,99,1,20170119,20170219,0
8,Np7r2ZaqVJ9oq35aci54ErdRp8xnrM+PZt8TMaq0xuQ=,38,30,149,149,0,20170120,20170219,0
9,GrPpcJke52rUOmgRwMUQGOh5MH7ADI+AbrsXepC2hx8=,40,30,149,149,1,20170120,20170219,0


In [41]:
transactions['payment_method_id'].value_counts()

41    11526454
40     2225283
38     1703590
39     1466655
37     1007689
36      855115
34      731539
35      541399
33      411164
31      252342
30      160957
32      146481
29      113885
28       95733
27       62525
23       42386
19       32073
20       28278
21       22883
22       20130
24       16196
18       16177
25       13780
14       13621
16       11064
17        7437
13        6571
26        4591
12        3834
11        2129
15        1479
10        1326
7         1094
8          657
5          474
6          466
3          210
2           52
4           15
1           12
Name: payment_method_id, dtype: int64

In [42]:
transactions['payment_plan_days'].value_counts()

30     18956290
0        870124
31       766608
7        577639
195      110234
410       80139
180       52272
10        38216
100       24154
90        12310
395       10790
120       10007
60         7167
14         6365
200        5838
360        5486
1          4759
400        1856
450        1271
240        1088
45         1082
21          869
35          825
70          587
80          342
99          339
365         312
270         190
66          175
110         159
230         103
2            67
3            59
15           21
415           1
425           1
95            1
Name: payment_plan_days, dtype: int64

In [43]:
transactions['plan_list_price'].value_counts()

149     12536656
99       4853433
0        1498544
129      1144459
180       682533
150       382860
894       109879
100        80285
1788       80066
536        43506
119        32280
480        22977
1599       11463
477        10847
35          7410
799         6264
300         5828
120         5821
1200        5486
298         5272
930         5082
447         4348
134         3743
450         1455
500         1185
50          1051
699          915
1000         750
124          598
350          587
131          374
105          348
400          342
70           250
143          143
2000         118
1            113
1150         103
126           99
596           70
10            67
15            50
210           26
265           21
1520          13
800           11
30             6
760            3
600            3
1825           2
44             1
Name: plan_list_price, dtype: int64

In [44]:
transactions['actual_amount_paid'].value_counts()

149     12460832
99       4855208
0        1196876
129      1174670
180       680058
150       396210
119       355083
894       113452
1788       83991
100        80969
536        44805
480        23170
1599       12086
477        10847
35          7892
799         6465
300         5996
1200        5526
298         5272
930         5121
120         4948
134         4807
447         4375
450         1501
500         1211
50          1056
699          913
1000         783
350          606
124          598
131          572
105          374
400          351
70           250
143          143
2000         122
1            113
1150         105
127           99
596           76
10            67
15            50
210           26
265           21
1520          13
800           11
30             6
41             5
760            4
600            3
1825           2
1799           1
849            1
1800           1
45             1
1802           1
890            1
Name: actual_amount_paid, dtype

In [45]:
transactions['is_auto_renew'].value_counts()

1    18357950
0     3189796
Name: is_auto_renew, dtype: int64

In [46]:
transactions["is_cancel"].value_counts()

0    20690895
1      856851
Name: is_cancel, dtype: int64

### 通过上面统计我们可以发现：
1. 一共21547746条数据，没有缺失数据
2. payment_method_id代表用户付款方式，编码为1-41整数，其中41数量最多
3. payment_plan_days代表用户计划买会员的天数，30占了绝大多数，最多450天，也有个别天数的个数只有1个，考虑为异常值
4. 计划购买的价格和实际付款价格可以一起比较以后和is_cancel字段对比出异常值
5. is_cancel字段代表用户是否取消了此次交易，绝大多数用户进行了交易
6. is_auto_renew代表用户是否自动续费，发现大多数（85%）选择，考虑为忠实用户
7. transaction_date 和 membership_expire_time 是预测用户是否流失的重要数据（但是存在异常值，需要进行清洗）

## 3.4 user_log.csv(取前500W条数据分析)

In [48]:
user_log1= pd.read_csv("G:\PythonProjects\KaggleOrOthersJourney\WSDM-KKBox'sChurnPredictionChallenge\data/user_log_split_1.csv")

In [49]:
user_log1.head(10)

,Unnamed: 0,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150513,0,0,0,0,1,1,280.335
1,1,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150709,9,1,0,0,7,11,1658.948
2,2,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150105,3,3,0,0,68,36,17364.956
3,3,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150306,1,0,1,1,97,27,24667.317
4,4,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150501,3,0,0,0,38,38,9649.029
5,5,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150702,4,0,1,1,33,10,10021.520
6,6,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150830,3,1,0,0,4,7,1119.555
7,7,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20151107,1,0,0,0,4,5,938.022
8,8,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20160110,2,0,1,0,11,6,3004.068
9,9,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20160316,9,3,4,1,67,50,18257.661


In [50]:
user_log1['msno'].value_counts()

Ggc/gTKNdredibcTbefQyg4X+K0xzf+dOT6DZLSz1Pc=    16
2i3h2MWfP04UC3hz/FDSzRzcyw48aNdShCHtfm/Lr0Y=    16
bOA0edn9Gwh9s/JPZtpGtCb4DgMRrF43Bd534D5ohr4=    16
Tqzjw9l4PHMd8YZS6xQZ+iZUkPGuGnwJ/hzMdil3vwE=    16
q7Jqv5rw7xj7z+BXIYtl0THvWq9YWrm9OPYxGmYEw44=    16
/CVDaFwhM22vyBGpSe2SWjboK3Ev8RjEYJoG4IROptw=    16
nOL4uZfxlCzlheeKMzB2dvkHAoqhcwiH9DusBVsKYpY=    16
b+vVKlmj9tC87kCx5WEuoqzYfkJ9lF2VcxZ/xc2n8DI=    16
VHmfbiayMC4tHiaEdZbc1X9AfDjTVNHTPfaap0jgbL4=    16
yrSI1R84j6PXO98X/vQCRYReLq0ejQBV4aD8DPTOyD4=    16
EbLRtRgjBTVWIVZ8o3pN1mGiLx2uFUR9eqHXq3Msjx8=    16
G2EHv+ytivQf85iCENgqYVNm1TLYw75vRz/O3UG9Wa4=    16
RsAloWpyqGMxEkrXAF7jrt20YhKdt0JKXkFjcEykd5k=    16
LKgp9ml68aUaiPDsgQlpQjXau04xoaEkqlbDRw0nkCI=    16
xLcj11lt7hQVMA27knqRIhPahTL5qZdxinLx1L9HSAc=    16
r765clYqHI9rz8fHGHsxjNPwDF5BAs/aLtjGmC38ZsE=    16
McQUruuNhVyoLRkQLdaNqSv6P3eXBDhvggoogBbBSJ8=    16
3KEa/SEny/JcEGLpOq6AjU99ZTeiJuNYxZXJilwcsXA=    16
brx5SumWTjRLOLtaGzlE9aYfPbD2kBg0YhLb9nSnLCk=    16
lziNCixECTWIf0FeXvDn3gJZWE9epx+

In [51]:
user_log1['num_25'].value_counts()

0       1303977
1        790943
2        513791
3        371125
4        284223
5        224318
6        182078
7        151545
8        127667
9        107116
10        91914
11        79387
12        69130
13        60486
14        53801
15        47643
16        42468
17        38079
18        33732
19        30794
20        27431
21        25033
22        22973
23        20762
24        19007
25        17504
26        16039
27        14614
28        13503
29        12594
         ...   
571           1
575           1
577           1
322           1
582           1
587           1
588           1
592           1
593           1
552           1
548           1
546           1
527           1
509           1
512           1
514           1
519           1
520           1
521           1
522           1
528           1
545           1
531           1
533           1
534           1
332           1
538           1
541           1
543           1
2515          1
Name: num_25, Length: 59

In [52]:
user_log1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 10 columns):
Unnamed: 0    int64
msno          object
date          int64
num_25        int64
num_50        int64
num_75        int64
num_985       int64
num_100       int64
num_unq       int64
total_secs    float64
dtypes: float64(1), int64(8), object(1)
memory usage: 381.5+ MB


In [53]:
user_log1.count()

Unnamed: 0    5000000
msno          5000000
date          5000000
num_25        5000000
num_50        5000000
num_75        5000000
num_985       5000000
num_100       5000000
num_unq       5000000
total_secs    5000000
dtype: int64

In [54]:
user_log1.describe()

,Unnamed: 0,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
count,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06
mean,2.500000e+06,2.015739e+07,6.505548e+00,1.641790e+00,1.016455e+00,1.128660e+00,3.075180e+01,3.011551e+01,-1.510788e+12
std,1.443376e+06,6.180511e+03,1.413799e+01,4.292380e+00,2.169558e+00,2.998862e+00,4.285198e+01,3.381988e+01,1.181791e+14
min,0.000000e+00,2.015010e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,-9.223372e+15
25%,1.250000e+06,2.015082e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,8.000000e+00,1.892287e+03
50%,2.500000e+06,2.016031e+07,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.700000e+01,1.900000e+01,4.628113e+03
75%,3.749999e+06,2.016091e+07,7.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,3.800000e+01,4.000000e+01,1.026891e+04
max,4.999999e+06,2.017023e+07,2.515000e+03,8.620000e+02,4.700000e+02,1.515000e+03,1.946200e+04,2.027000e+03,9.223372e+15


In [55]:
user_log1['num_50'].value_counts()

0      2398292
1      1138083
2       554561
3       300844
4       178667
5       111636
6        74110
7        51566
8        37246
9        27436
10       20851
11       16060
12       12628
13       10344
14        8328
15        6958
16        5807
17        4846
18        4065
19        3564
20        3123
21        2729
22        2488
23        2123
24        1854
25        1635
26        1476
27        1290
28        1177
29        1106
        ...   
270          1
172          1
210          1
211          1
212          1
204          1
214          1
215          1
755          1
203          1
221          1
223          1
194          1
225          1
229          1
231          1
187          1
234          1
238          1
240          1
241          1
186          1
244          1
245          1
247          1
248          1
249          1
252          1
180          1
862          1
Name: num_50, Length: 276, dtype: int64

In [56]:
user_log1['num_75'].value_counts()

0      2716689
1      1183618
2       515550
3       249611
4       130562
5        72376
6        42898
7        26081
8        16635
9        11250
10        7922
11        5693
12        3909
13        2955
14        2336
15        1766
16        1383
17        1105
18         937
19         703
20         662
21         578
22         465
23         383
24         347
25         307
26         263
27         252
28         210
29         199
        ...   
235          1
164          1
267          1
285          1
295          1
351          1
166          1
140          1
162          1
161          1
106          1
112          1
113          1
114          1
122          1
124          1
130          1
131          1
132          1
133          1
144          1
146          1
149          1
150          1
152          1
153          1
154          1
156          1
158          1
470          1
Name: num_75, Length: 172, dtype: int64

In [57]:
user_log1['num_985'].value_counts()

0       2657484
1       1173724
2        521045
3        260393
4        141492
5         81775
6         49591
7         31758
8         20888
9         14306
10         9848
11         7216
12         5410
13         3974
14         3162
15         2415
16         1945
17         1542
18         1341
19         1119
20          885
21          752
22          672
23          529
24          471
25          413
26          377
28          316
27          315
29          289
         ...   
215           1
619           1
1148          1
1162          1
216           1
179           1
209           1
206           1
1265          1
152           1
157           1
162           1
166           1
175           1
177           1
180           1
182           1
183           1
184           1
185           1
186           1
189           1
190           1
191           1
192           1
193           1
194           1
199           1
201           1
1515          1
Name: num_985, Length: 2

In [58]:
user_log1['num_100'].value_counts()

1       227339
4       189062
0       183335
2       182046
3       179063
5       166562
6       152506
7       152236
8       144980
9       134939
10      129595
11      124709
12      117678
13      110715
14      106199
15       99986
16       95692
17       90889
18       85823
19       81404
20       78381
21       74327
22       70586
23       66556
24       64466
25       61628
26       58546
27       55682
28       53420
29       51113
         ...  
849          1
845          1
841          1
839          1
873          1
834          1
833          1
831          1
822          1
819          1
817          1
868          1
874          1
921          1
900          1
920          1
914          1
913          1
911          1
910          1
904          1
898          1
875          1
897          1
895          1
893          1
889          1
883          1
880          1
3830         1
Name: num_100, Length: 971, dtype: int64

In [59]:
user_log1['num_unq'].value_counts()

1       231057
2       184550
4       170766
3       166541
5       160130
6       145713
7       142087
8       140381
9       133094
10      130399
11      125826
12      120433
13      115121
14      109633
15      105111
16      100326
17       95964
18       92228
20       89030
19       88330
21       82355
22       78264
23       74387
24       71572
25       68970
26       65825
27       63199
28       60351
29       58602
30       56906
         ...  
666          1
556          1
431          1
658          1
652          1
435          1
501          1
641          1
637          1
633          1
631          1
627          1
626          1
625          1
623          1
444          1
610          1
609          1
607          1
446          1
603          1
599          1
595          1
591          1
455          1
577          1
565          1
462          1
558          1
2027         1
Name: num_unq, Length: 626, dtype: int64

In [60]:
user_log1['total_secs'].value_counts()

265.561      554
265.000      427
287.328      233
30.041       233
229.590      231
215.144      228
234.000      227
216.000      223
229.000      221
265.590      221
219.000      209
215.000      207
249.888      205
232.307      198
258.000      196
287.000      185
259.000      180
245.000      179
231.000      178
235.000      177
265.508      175
263.000      171
211.409      171
210.000      170
233.000      169
219.324      169
227.000      168
205.000      167
249.000      165
238.000      164
            ... 
30762.197      1
5668.713       1
4334.021       1
15178.157      1
16564.610      1
18974.889      1
3165.026       1
2933.168       1
3623.289       1
8609.903       1
11451.748      1
6923.745       1
34984.297      1
2890.832       1
16567.610      1
458.310        1
459.690        1
34984.797      1
19625.069      1
15179.782      1
8606.347       1
6932.380       1
2368.569       1
8609.528       1
1787.636       1
15649.335      1
16564.860      1
1270.797      

In [61]:
user_log1['num_25'].mean()

6.5055478

In [62]:
user_log1['num_50'].mean()

1.6417898

In [63]:
user_log1['num_75'].mean()

1.016455

In [64]:
user_log1['num_985'].mean()

1.12866

In [65]:
user_log1['num_100'].mean()

30.751797

In [66]:
user_log1['num_unq'].mean()

30.115508

In [67]:
user_log1['total_secs'].mean()

-1510788331458.5325

In [71]:
user_log1[user_log1['total_secs'] < 0]['total_secs']

4497      -9.223372e+15
5730      -9.223372e+15
11600     -9.223372e+15
11750     -9.223372e+15
12969     -9.223372e+15
13894     -9.223372e+15
23092     -9.223372e+15
40018     -9.223372e+15
42097     -9.223372e+15
42552     -9.223372e+15
48791     -9.223372e+15
56747     -9.223372e+15
58867     -9.223372e+15
59009     -9.223372e+15
82681     -9.223372e+15
83785     -9.223372e+15
86968     -9.223372e+15
87341     -9.223372e+15
96079     -9.223372e+15
99728     -9.223372e+15
103915    -9.223372e+15
125214    -9.223372e+15
134092    -9.223372e+15
137327    -9.223372e+15
139484    -9.223372e+15
145054    -9.223372e+15
146364    -9.223372e+15
156848    -9.223372e+15
159626    -9.223372e+15
161246    -9.223372e+15
               ...     
4868025   -9.223372e+15
4877040   -9.223372e+15
4877718   -9.223372e+15
4883460   -9.223372e+15
4891875   -9.223372e+15
4896192   -9.223372e+15
4898024   -9.223372e+15
4904866   -9.223372e+15
4909312   -9.223372e+15
4917251   -9.223372e+15
4923670   -9.223

### 通过统计发现：
1. 用户的行为数据有的存在多条，有的存在一条，肯定也有用户不存在（需要比对，没有任何行为的用户是流失潜在用户）
2. 用户听歌时长(秒)存在负值，需要清洗
3. 用户听某首歌时间占歌曲总时间区间分别为[<25%, 25% - 50%, 50% - 70%, 70% - 98.5%, >98.5%]，每个段中位数为1-2之间，除了num_100的为17
4. 日志时间段为2015-01 ~ 2017-02
5. num_unq代表了那天用户播放了多少不同的歌，中位数为30

### 3.5 sample_submission_zero.csv

In [72]:
submission_data = pd.read_csv("G:\PythonProjects\KaggleOrOthersJourney\WSDM-KKBox'sChurnPredictionChallenge\data\sample_submission_zero.csv")

In [77]:
submission_data.describe()

,is_churn
count,970960.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


## 4. 特征可视化

In [72]:
## 4.缺失值
除了用户的性别，其他没有缺失值